In [8]:
import gdown
import pulp as p

import pandas as pd

url = "https://drive.google.com/uc?id=15BPH7-3GGWBfXPJQ3stkT6SHQECbT-pt"
output = "shifts.csv"
gdown.download(url, output, quiet=False)

df = pd.read_csv("shifts.csv", index_col=0)

Downloading...
From: https://drive.google.com/uc?id=15BPH7-3GGWBfXPJQ3stkT6SHQECbT-pt
To: /Users/ethansu/Desktop/Grandeur_Log/integer_programming/shifts.csv
100%|██████████| 298/298 [00:00<00:00, 288kB/s]


In [2]:
df

,Shift 1,Shift 2,Shift 3,Shift 4,Workers Required
Time Windows,,,,,
6:00 – 9:00,X,NaN,NaN,X,55.0
9:00 – 12:00,X,NaN,NaN,NaN,46.0
12:00 – 15:00,X,X,NaN,NaN,59.0
15:00 – 18:00,NaN,X,NaN,NaN,23.0
18:00 – 21:00,NaN,X,X,NaN,60.0
21:00 – 24:00,NaN,NaN,X,NaN,38.0
24:00 – 3:00,NaN,NaN,X,X,20.0
3:00 – 6:00,NaN,NaN,NaN,X,30.0
Wage rate per 9h shift ($),135,140,190,188,NaN


In [3]:
df = df.fillna(0).applymap(lambda x: 1 if x == "X" else x)

a = df.drop(index=["Wage rate per 9h shift ($)"], columns=["Workers Required"]).values

a

array([[1, 0, 0, 1],
       [1, 0, 0, 0],
       [1, 1, 0, 0],
       [0, 1, 0, 0],
       [0, 1, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 1],
       [0, 0, 0, 1]], dtype=object)

In [5]:
# number of shifts
n = a.shape[1]

# number of time windows
T = a.shape[0]

# number of workers required per time window
d = df["Workers Required"].values

# wage rate per shift
w = df.loc["Wage rate per 9h shift ($)", :].values.astype(int)

w

array([135, 140, 190, 188,   0])

In [9]:
# Decision variables
y = p.LpVariable.dicts("num_workers", list(range(n)), lowBound=0, cat="Integer")

In [10]:
# Create problem
prob = p.LpProblem("scheduling_workers", p.LpMinimize)

In [11]:

prob += p.lpSum([w[j] * y[j] for j in range(n)])

In [12]:
for t in range(T):
    prob += p.lpSum([a[t, j] * y[j] for j in range(n)]) >= d[t]

In [13]:
prob.solve()
print("Status:", p.LpStatus[prob.status])

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/envs/friday/lib/python3.8/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/gr/kkqqzv6532ldlp5jhz8v3bmc0000gn/T/fa253b124a0c4107ae8fe37e65eeab33-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/gr/kkqqzv6532ldlp5jhz8v3bmc0000gn/T/fa253b124a0c4107ae8fe37e65eeab33-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 13 COLUMNS
At line 38 RHS
At line 47 BOUNDS
At line 52 ENDATA
Problem MODEL has 8 rows, 4 columns and 12 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 22290 - 0.00 seconds
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cbc3007W No integer variables - nothing to do
Cuts at root node changed objective from 22290 to -1.79769e+308
Probing was tried 0 times and created 0 cuts of which 0 

In [14]:
for shift in range(n):
    print(
        f"The number of workers needed for shift {shift} is {int(y[shift].value())} workers"
    )

The number of workers needed for shift 0 is 46 workers
The number of workers needed for shift 1 is 23 workers
The number of workers needed for shift 2 is 38 workers
The number of workers needed for shift 3 is 30 workers
